# **RSA** [Prepared by Jevin Evans and Muhammad Ismail]
---

**RSA** (Rivest-Shamir-Adleman) is a popular public-key cryptographic system used to encrypted data. RSA gains its strength from large prime numbers, modular exponentiation, and modular multiplicative inverse.

The following will teach you how key generation, encryption, and decryption work in RSA. This example will use a smaller number for demonstration purposes. Focus on the operations and not specifically on the code implementation.

##**Instructions** 
Run in order by hitting the play button next to each code box, or in the <u>menu bar</u>, select **Runtime->Run All** to run the whole lab.

In [ ]:
#@title Plaintext Message
#@markdown Insert a message that you want to encrypt or use the example

PLAINTEXT = "This is an example of RSA Public Key Cryptography." #@param {type:"string"}

from random import randint
from math import pow, sqrt

# Returns the boolean of whether a number is prime or not
def isPrime(p):
  if p <= 1: return False
  if p == 2: return True

  for x in range(2, int(sqrt(p))+1):
    if p % x == 0: return False

  return True; 

def gcd(a, b): 
    if a == 0:
        return b
    return gcd(b%a, a)

#Important Functions
---

These are some critical operations and functions that will be used later on in the code.

- **Modular Multiplicative Inverse** - given a number *a* and a modulo *m*, the modular multiplicative inverse of *a* is *b* such that $a*b \equiv 1 ($mod $m) $

- **Coprime Numbers** - numbers *a* and *b* are coprime to each other if the greatest common divisor between *a* and *b* is 1 gcd($a, b$) $\equiv 1$.


In [ ]:
# Returns the Modular Inverse of a number given a modulos number
def modInv(a, modulo):
  for b in range(1, modulo):
    if (a * b) % modulo == 1:
      return b
  return 0

# Returns the boolean on whether two numbers are coprime to each other
def isCoprime(a, b):
  return gcd(a,b) == 1

# Key Generation
---

The key generation process for RSA is simple but mathematically strong enough to be unsolvable when large numbers are used. For a user to generate their private and public keys in this system they perform the following steps: 

1. First, choose two large, distinct prime numbers, **p** and **q**. 
1. Compute **n** by multiplying **p** and **q**.
1. Compute **phi**, the Euler Totient of **n**.
1. Choose a random **d** (private key) such that **d** is coprime to **phi**
1. Compute **e**, which is the modular inverse of **d**, such that **d*e $\equiv$ 1 mod phi**
1. Distribute public key as (n, e)

##"Large Primes" - p and q

**P** and **Q** are large, prime numbers chosen randomly to begin the key generation process. These values are kept secret, and the larger they are, the more secure pair you can generate.

For this example, smaller numbers will be used for faster computations.


In [ ]:
# Randomly choose p and validates prime number
p = randint(10, 250)
while not isPrime(p):
  p = randint(10, 500)

# Randomly choose q, validate not prime, and not equal to p
q = randint(10, 250)
while not isPrime(q) and q != p:
  q = randint(10, 500)

print(f"p: {p}\nq: {q}")

p: 31
q: 23


## Public Modulos - n
The **n** value is the result of multiplying the secret **p** and **q** numbers. This number is used for modulo operations in RSA for encrypting and decrypting messages. The larger the **n** value, the harder it is to factor the **p** and **q** values used.

**n** = **p** \* **q**

In [ ]:
n = p*q

print(f"n: {n}")

n: 713


## Eulers Totient - phi
Using the generated **n** value, **phi** is the [Euler Totient](https://en.wikipedia.org/wiki/Euler%27s_totient_function) of **n**, the number of positive coprime integers to n. Phi is used to generate the private key. This value is kept secret because it is used to generate the private key.   
 
 
Since **p** and **q** are prime numbers, using math theory, there is a more straightforward method to compute the totient.

**phi = (p-1)*(q-1)**


In [ ]:
phi = (p-1)*(q-1)
print(f"Euler Totient (phi): {phi}")

Euler Totient (phi): 660


## Private Key - d
The private key, **d**, is kept secret to the user. This number is chosen at random such that $1 < d < phi$. Suppose **d** is chosen as a prime number from $max(p,q) < d < phi$, then it will be more secure using the power of primes. Security in the sense of it becomes harder for a cryptanalyst to discover the private key.

In [ ]:
# Random select private key
Alice_privateKey = randint(max(p,q)+1, phi-1)

# Verify private key will work
while not isCoprime(Alice_privateKey, phi) and not isPrime(Alice_privateKey):
  Alice_privateKey = randint(max(p,q)+1, phi-1)

print(f"Alice Private Key (d): {Alice_privateKey}")

Alice Private Key (d): 577


## Public Key - e
The public key, **e**, is computed as the modular multiplicative inverse of the private key, **d**. Since the public key is generated using the phi value and not **n**, it is harder for a cryptoanalyst to crack.

$e \equiv d^{-1} ($mod $phi)$

$e*d \equiv 1 ($mod $phi)$


In [ ]:
Alice_publicKey = modInv(Alice_privateKey, phi)

print(f"Alice Public Key (e): {Alice_publicKey}")

print(f"\nVerification of Modular Inverse: d*e == 1 (mod phi)\n\t{Alice_privateKey}*{Alice_publicKey} % {phi} == 1 -> {Alice_privateKey*Alice_publicKey%phi == 1}")

Alice Public Key (e): 493

Verification of Modular Inverse: d*e == 1 (mod phi)
	577*493 % 660 == 1 -> True


## Distribute Public Key

Once the keys are generated, the public key is shared as a pair with the user's **e** (public key) and the **n** value used for modulo. Remember the **d** (private key), **p**, **q**, and **phi** are never shared.

In [ ]:
ALICE = (n , Alice_publicKey)
print(f"Alice Distributes Public Key Pair (n, e): {ALICE}")

Alice Distributes Public Key Pair (n, e): (713, 493)


# Encryption
---

The process of encrypting messages in RSA is performed using modular exponentiation. Messages can be encrypted with either a person's public or private key. The key used depends on who the sender is. 

In this lab, we have generated Alice's keys. If Alice wants to send a message to Bob, the message will be encrypted with Alice's private key, and Bob will decrypt it with Alice's public key. Vice versa, if Bob sends a message to Alice, he will encrypt it with Alice's public key, and Alice will decrypt it with her private key.


The equation for encryption is as follows for Bob sending Alice a message:

$C \equiv m^{e} ($mod $n)$, where

- **C** is the ciphertext
- **m** is the message being encrypted (numerical format)
- **e** Alice's public key
- **n** Alice's n value

In [ ]:
print("\nPLAINTEXT (ASCII):", PLAINTEXT)
print("\nPLAINTEXT (Decimal):\n", [ord(i) for i in PLAINTEXT])
print("\n\tExample: C = "+str(ord(PLAINTEXT[0]))+"**"+str(Alice_publicKey)+" mod "+str(n)+" = "+str(ord(PLAINTEXT[0])**Alice_publicKey % n))

# Encryptes the whole message
ciphertext = [ord(i)**Alice_publicKey % n for i in PLAINTEXT]
print("\nCiphertext (Decimal):\n", ciphertext)

# Joins the Message into a single string
ciphertext = ''.join([chr(i) for i in ciphertext])
print("\nCiphertext (ASCII):", ciphertext)


PLAINTEXT (ASCII): This is an example of RSA Public Key Cryptography.

PLAINTEXT (Decimal):
 [84, 104, 105, 115, 32, 105, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 101, 32, 111, 102, 32, 82, 83, 65, 32, 80, 117, 98, 108, 105, 99, 32, 75, 101, 121, 32, 67, 114, 121, 112, 116, 111, 103, 114, 97, 112, 104, 121, 46]

	Example: C = 84**493 mod 713 = 106

Ciphertext (Decimal):
 [106, 579, 141, 230, 94, 141, 230, 94, 126, 127, 94, 140, 494, 126, 283, 603, 399, 140, 94, 516, 204, 94, 72, 642, 148, 94, 640, 167, 315, 399, 141, 130, 94, 476, 140, 131, 94, 408, 22, 131, 603, 139, 516, 226, 22, 126, 603, 579, 131, 368]

Ciphertext (ASCII): jɃæ^æ^~^Ǯ~ěɛƏ^ȄÌ^Hʂ^ʀ§ĻƏ^ǜ^ƘɛȄâ~ɛɃŰ


# Decryption
---

Decryption is done the same way, just using the opposite key. In this case, Bob sent the message to Alice, so Alice will use her private key to decrypt the message.

The equation for decryption is as follows for Alice decrypting the ciphertext:

$m \equiv C^{d} ($mod $n)$, where

- **m** is the ciphertext (numerical format)
- **C** is the ciphertext
- **d** Alice's private key
- **n** Alice's modulo n value

In [ ]:
print("\nCiphertext (ASCII):", ciphertext)
print("\nCiphertext (Decimal):", [ord(i) for i in ciphertext])
print("\n\tExample: M = "+str(ord(ciphertext[0]))+"**"+str(Alice_privateKey)+" mod "+str(n)+" = "+str(ord(ciphertext[0])**Alice_privateKey % n))
plaintext = [ord(i)**Alice_privateKey % n for i in ciphertext]
print("\nPlaintext (Decimal):\n", plaintext)
plaintext = ''.join([chr(i) for i in plaintext])
print("\n\nPlaintext (ASCII):", plaintext)


Ciphertext (ASCII): jɃæ^æ^~^Ǯ~ěɛƏ^ȄÌ^Hʂ^ʀ§ĻƏ^ǜ^ƘɛȄâ~ɛɃŰ

Ciphertext (Decimal): [106, 579, 141, 230, 94, 141, 230, 94, 126, 127, 94, 140, 494, 126, 283, 603, 399, 140, 94, 516, 204, 94, 72, 642, 148, 94, 640, 167, 315, 399, 141, 130, 94, 476, 140, 131, 94, 408, 22, 131, 603, 139, 516, 226, 22, 126, 603, 579, 131, 368]

	Example: M = 106**577 mod 713 = 84

Plaintext (Decimal):
 [84, 104, 105, 115, 32, 105, 115, 32, 97, 110, 32, 101, 120, 97, 109, 112, 108, 101, 32, 111, 102, 32, 82, 83, 65, 32, 80, 117, 98, 108, 105, 99, 32, 75, 101, 121, 32, 67, 114, 121, 112, 116, 111, 103, 114, 97, 112, 104, 121, 46]


Plaintext (ASCII): This is an example of RSA Public Key Cryptography.


# Conclusion
---

Now that you have completed this lab, you should understand how the RSA cryptographic scheme works.

# Resources
1. https://en.wikipedia.org/wiki/RSA_(cryptosystem) 
1. https://en.wikipedia.org/wiki/Euler%27s_totient_function 
1. http://www.blackwasp.co.uk/Coprime.aspx 
1. https://sites.math.washington.edu/~morrow/336_09/papers/Yevgeny.pdf 
1. https://www.geeksforgeeks.org/multiplicative-inverse-under-modulo-m/ 
1. http://people.csail.mit.edu/rivest/Rsapaper.pdf 